In [1]:
import pandas as pd
from datetime import datetime
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.dialects.postgresql import insert


from models import Game
import date_utils as date_mng
from data_manager import DataManager
data_manager = DataManager()
self = data_manager

In [2]:
teams = self.query_teams()
season_type = "Playoffs"
season = "2023-24"
team = teams[1]
print(teams)


[<models.Team object at 0x000001DD7E956BD0>, <models.Team object at 0x000001DD7E956DE0>, <models.Team object at 0x000001DD7E956F90>, <models.Team object at 0x000001DD7E956330>, <models.Team object at 0x000001DD7E956480>, <models.Team object at 0x000001DD7E956BA0>, <models.Team object at 0x000001DD7E9577A0>, <models.Team object at 0x000001DD7E955700>, <models.Team object at 0x000001DD7E956000>, <models.Team object at 0x000001DD7E956210>, <models.Team object at 0x000001DD7E9560F0>, <models.Team object at 0x000001DD7E955970>, <models.Team object at 0x000001DD7E955AC0>, <models.Team object at 0x000001DD7E955C70>, <models.Team object at 0x000001DD7E955430>, <models.Team object at 0x000001DD7E9556D0>, <models.Team object at 0x000001DD7E954D10>, <models.Team object at 0x000001DD7E955AF0>, <models.Team object at 0x000001DD7E955E50>, <models.Team object at 0x000001DD7E954AD0>, <models.Team object at 0x000001DD7E954C20>, <models.Team object at 0x000001DD7E9552B0>, <models.Team object at 0x000001

In [3]:
games = self.pull_all_games_from_season(season, season_type)



e:\coding_projects\nba_01\data_manager.py:216: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games = pd.concat(all_games, axis=0, ignore_index=True)


In [4]:
#for game in games:
for _, game in games.iterrows():
    break

In [5]:
nba_game_id = game['GAME_ID']
adv_player_stats, adv_team_stats = self.pull_advanced_stats_for_game(nba_game_id)
trad_player_stats, trad_team_stats = self.pull_traditional_stats_for_game(nba_game_id)

In [6]:

db_game_id = self.sync_game(game, season, season_type)


In [7]:
trad_team_stats_ids = self.sync_trad_team_stats(trad_team_stats=trad_team_stats, db_game_id=db_game_id)

In [9]:
adv_team_stats_ids = self.sync_adv_team_stats(adv_team_stats=adv_team_stats, db_game_id=db_game_id)

In [10]:
trad_player_stats_ids = self.sync_trad_player_stats(trad_player_stats=trad_player_stats, db_game_id=db_game_id)

In [ ]:
adv_player_stats_ids = self.sync_adv_player_stats(adv_player_stats=adv_player_stats, db_game_id=db_game_id)